In [15]:
pip install optuna

Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [18]:
pip install graphviz

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import random
import os
import seaborn as sns



In [2]:
train = pd.read_csv('C:/Users/hop09/Downloads/open/train.csv')
train.head()

,ID,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,TRAIN_000,4,103,60,33,192,24.0,0.966,33,0
1,TRAIN_001,10,133,68,0,0,27.0,0.245,36,0
2,TRAIN_002,4,112,78,40,0,39.4,0.236,38,0
3,TRAIN_003,1,119,88,41,170,45.3,0.507,26,0
4,TRAIN_004,1,114,66,36,200,38.1,0.289,21,0


## 데이터전처리
- 혈액농도 mean값
- bmi mean값

In [3]:
#만약 평균값으로 대체하고 싶다면 이거 하기
mean_b = train["BloodPressure"].mean()
train["BloodPressure"].replace(0, mean_b)

mean_bmi = train["BMI"].mean()
train["BMI"].replace(0, mean_bmi)

0      24.0
1      27.0
2      39.4
3      45.3
4      38.1
       ... 
647    29.2
648    31.2
649    38.3
650    30.1
651    26.5
Name: BMI, Length: 652, dtype: float64

In [4]:
# # ID, 임신횟수, 피부두께 버려
train.drop(axis = 1, columns= ["ID", "Pregnancies", "SkinThickness"], inplace = True)
train.head()

,Glucose,BloodPressure,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,103,60,192,24.0,0.966,33,0
1,133,68,0,27.0,0.245,36,0
2,112,78,0,39.4,0.236,38,0
3,119,88,170,45.3,0.507,26,0
4,114,66,200,38.1,0.289,21,0


In [5]:
# 인슐린 107로 대체
train["Insulin"].replace(0,107, inplace = True)

In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
train_scaled = scaler.fit_transform(train.iloc[:,:-1]).round(4)

train_scaled_df = pd.DataFrame(train_scaled, columns=train.columns[:-1])
train_scaled_df['target'] = train['Outcome']
train_scaled_df.head()


,Glucose,BloodPressure,Insulin,BMI,DiabetesPedigreeFunction,Age,target
0,-0.5614,-0.4641,0.6484,-1.0660,1.5213,-0.0432,0
1,0.3771,-0.0467,-0.2868,-0.6636,-0.6686,0.2107,0
2,-0.2799,0.4749,-0.2868,0.9996,-0.6959,0.3799,0
3,-0.0609,0.9966,0.4063,1.7910,0.1272,-0.6357,0
4,-0.2173,-0.1511,0.7364,0.8253,-0.5350,-1.0588,0


### K fold 쓰기

In [7]:
from sklearn.model_selection import train_test_split, StratifiedKFold

x = np.array(train.values[:,-1:])
y = train.values[:,:-1]

# split 개수, 셔플 여부 및 seed 설정
stf = StratifiedKFold(n_splits = 5, shuffle = True, random_state= 0) 

In [8]:
import torch
data = torch.from_numpy(train_scaled_df.values).float()

x = data[:,:-1]
y = data[:, -1:]

print(x.shape, y.shape)

torch.Size([652, 6]) torch.Size([652, 1])


In [14]:
# n_epochs = 20000
# learning_rate = 1e-2
# print_interval = 1000

# Optuna - lgbm

In [31]:
from sklearn.metrics import accuracy_score
import optuna
import lightgbm as lgbm
from optuna import Trial
from optuna.samplers import TPESampler
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split


def objective(trial):
    params_lgbm = {
        'random_state' : 0,
        'early_stopping_round' : 50,
        'n_estimators' : trial.suggest_int('n_estimators', 50, 200),
        'max_depth': trial.suggest_int('max_depth', 5, 15),
        'num_leaves': trial.suggest_categorical('max_leaves', [2**i for i in range(4, 10)]),
        'subsample': trial.suggest_uniform('subsample', 0.0, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.0, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 10.0) 
        }

    for train_index, test_index in kfold.split(x): # for문으로 index순회하면서 train, test셋 뽑기

        X_train, X_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
            

        model = XGBClassifier(**params_xgb)
        model.fit(
            X_train, 
            y_train, 
            eval_set = [(X_train,y_train ), (X_test,y_test )],
            verbose=False,
        
        )
        xgb_pred = model.predict_proba(X_test)
        log_score = log_loss(y_test, xgb_pred)
        
    return log_score

In [32]:
sampler = TPESampler(seed=42)
study = optuna.create_study(
    study_name="xgb_parameter_opt",
    direction="minimize",
    sampler=sampler,
)
study.optimize(objective, n_trials=10)
print("Best Score:", study.best_value)
print("Best trial:", study.best_trial.params)

[I 2023-03-28 00:56:46,745] A new study created in memory with name: xgb_parameter_opt
C:\Users\hop09\AppData\Local\Temp\ipykernel_16060\2527784243.py:18: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 10.0),
C:\Users\hop09\AppData\Local\Temp\ipykernel_16060\2527784243.py:19: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 10.0),
C:\Users\hop09\AppData\Local\Temp\ipykernel_16060\2527784243.py:20: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/

Best Score: 0.6931471824645996
Best trial: {'n_estimators': 106, 'max_depth': 15, 'max_leaves': 512, 'min_child_weight': 181, 'reg_alpha': 0.023585940584142682, 'reg_lambda': 1.5320059381854043e-08, 'colsample_bytree': 0.9699098521619943, 'subsample': 0.8324426408004217, 'learning_rate': 0.11404616423235531}


In [42]:

model = XGBClassifier(**study.best_trial.params)
model.fit(x, y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9699098521619943, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.11404616423235531,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=15, max_leaves=512,
              min_child_weight=181, missing=nan, monotone_constraints=None,
              n_estimators=106, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

# 테스트 셋 전처리

test = test.drop(columns = ['ID'])
test_scaled = scaler.transform(test)

In [43]:
test = pd.read_csv('C:/Users/hop09/Downloads/open/test.csv')
test.head()

,ID,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,TEST_000,5,112,66,0,0,37.8,0.261,41
1,TEST_001,3,107,62,13,48,22.9,0.678,23
2,TEST_002,3,113,44,13,0,22.4,0.140,22
3,TEST_003,6,190,92,0,0,35.5,0.278,66
4,TEST_004,1,107,72,30,82,30.8,0.821,24


In [44]:
# 예측
test = pd.read_csv('C:/Users/hop09/Downloads/open/test.csv')
test.drop(columns = ["ID", "Pregnancies", "SkinThickness"], inplace = True)


In [45]:
test["Insulin"].replace(0,107, inplace = True)

In [46]:
mean_b = test["BloodPressure"].mean()
test["BloodPressure"].replace(0, mean_b)

mean_bmi = test["BMI"].mean()
test["BMI"].replace(0, mean_bmi)

0      37.800000
1      22.900000
2      22.400000
3      35.500000
4      30.800000
         ...    
111    27.500000
112    22.300000
113    36.400000
114    31.600000
115    32.246552
Name: BMI, Length: 116, dtype: float64

In [47]:
type(test)

pandas.core.frame.DataFrame

In [48]:
predict = model.predict(test)

In [49]:
predict

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0])

In [ ]:
# test_scaler = scaler.transform(test)

# test_set = torch.from_numpy(test_scaler).float()


c:\Users\hop09\Desktop\anaconda\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [ ]:
# test_set.shape

torch.Size([116, 7])

In [ ]:
# round(1.735, 0) 

2.0

In [ ]:
# test_loss = 0
# predict_ = []

# with torch.no_grad() :
# 		x_ = test_set

# 		for x_i in x_ :
# 				y_hat_i = model(x_i)
# 				predict_ += [y_hat_i.round()]


In [ ]:
# predict = [int(predict.item()) for predict in predict_]

### CSV로 저장

In [64]:
submit = pd.read_csv('C:/Users/hop09/Downloads/open/sample_submission.csv')

In [65]:
submit['Outcome'] = predict

In [66]:
submit.to_csv('C:/Users/hop09/Downloads/open/submit.csv', index = False)